In [1]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import numpy as np
from konlpy.tag import Okt
from tqdm import tqdm
import operator 

In [2]:
origin_data=pd.read_table(r'japan_fujimountain_origin.csv',sep=",")
origin_data.columns

Index(['Title', 'View', 'Score', 'Score2', 'View8', 'Time1'], dtype='object')

In [3]:
origin_data['Title'] = origin_data['Title'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","",regex=True)
origin_data = origin_data.dropna(how = 'any')
print(len(origin_data))

453


In [4]:
## 불용어 정의
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','이다','가다','오다','가면','일본','그것','되다','니다']

okt = Okt()

## 불용어를 제거하여 X_train에 저장
document = []
for sentence in tqdm(origin_data['Title']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    document.append(stopwords_removed_sentence)


100%|████████████████████████████████████████████████████████████████████████████████| 453/453 [00:07<00:00, 60.47it/s]


In [5]:
detokenized_doc = []
for i in range(len(origin_data)):
    t = ' '.join(document[i])
    detokenized_doc.append(t)

origin_data['Title'] = detokenized_doc

In [51]:
 detokenized_doc

['여름 비 후지산 보다 눈 보지 못 후지산 온통 푸르다 또 다른 면 아름답다 거기 도착 것 매우 편리하다 자동차 요금 비밀리 조금 비싸다 날씨 좋다 구매 티켓 있다 엘리베이터 타고 호수 전망 을 보고 산 전망 을 보다 정원 을 보다 갈다 수 있다 나무 수 있다',
 '처음 후지산 을 이렇게 가까이 서 보다 자연 스럽게 목소리 나오다 정도 로 아름답다 산 근처 에서 날씨 변하다 쉬다 때문 낮 에는 구름 숨다 경우 많다 아침 일찍 일어나서 보다 후지산 깨끗하다 감동 을 줄 정도 로 아름답다 또 꼭 방문 하고 싶다 곳',
 '후지산 국내 에서 가장 유명하다 관광 명소 을 여행 때 보통 후지산 아래 에서 하룻밤 을 지내다 온천 몸 을 담그다 후지산 을 올라가다 수 있다 후지산 활화산 최근 모니터링 단계 있다 분화',
 '높다 광고 도달 것 믿다 수 없다 정도 로 놀라다 후지산 에서 안개 자욱하다 않다 하루 날씨 좋다 적극 권장',
 '월 월 날씨 좋다 치산 매우 아름답다 계절 기간 최고 로 간주',
 '사계절 내내 언제 든지 아름답다 곳 새벽 감성 좋다',
 '을 방문 않다 오랜 시간 이제 후지산 가기 정말 좋다 시간 너무 아름답다 매혹 적',
 '이르다 아침 매우 권장 참고 로이 사진 오전 시',
 '경치 아름답다 풍경 재미 재미 비용 효과 저렴하다 가격',
 '후지산 랜드 마크 정말 아름답다 아름답다',
 '로프웨이 타고 전망 대로 자다 보이다 후지산 너무 아름답다',
 '가을 후지 아름답다 엿보다 모든 것 너무 숨 막히다 것 곧 달다 싶다',
 '구름 위 살다 것 같다',
 '세계 적 유명하다 풍경 말 않다 카드 펀치 몇 가지 포인트 다른 스타일 을 가지 고 있다',
 '도착 후지산 사진 을 찍다 오지 않다 도착 않다 것 간주',
 '필수 카드 명소 사진 촬영 매우 적합 눈 아름답다',
 '후지산 눈 녹지 않다 멀리 서 볼 수 있다',
 '시즈오카현 에서 후지산 을 보다 것 정말 아름답다',
 '볼 만 가치 있다 곳 을 이해',
 '년 전 후지산 아름답다 전망

In [59]:
# TF-IDF 행렬javascript:page(159467)
# vectorizer = TfidfVectorizer(stop_words=["스럽다"], max_features= 2000, # 상위 1,000개의 단어를 보존 
# max_df = 0.2, smooth_idf=True)
vectorizer=TfidfVectorizer(analyzer = 'word', min_df=2)

X = vectorizer.fit_transform(origin_data['Title'])

# TF-IDF 행렬의 크기 확인
print('TF-IDF 행렬의 크기 :',X.shape) ##(토픽 개수, 단어 개수)
word_cnt=X.shape[1]

TF-IDF 행렬의 크기 : (453, 991)


In [60]:
#토픽 모델링(Topic Modeling)
#n_components는 토픽의 개수
svd_model = TruncatedSVD(n_components=word_cnt, algorithm='randomized', n_iter=7, random_state=122)
svd_model.fit(X)
len(svd_model.components_)

#‘속성-키워드 명명화(naming)’ 작업
##get_feature_names() 대신 get_feature_names_out() 사용
terms = vectorizer.get_feature_names_out() # 단어 집합. 1,000개의 단어가 저장됨.

##topic은 svd_model topic.argsort()하게 되면 오름차순으로 정렬되며 -n슬라이싱으로 큰 숫자 가져옴
##topic.argsort()의 반환값은 해당 숫자의 원래 인덱스

tag_idx={}
tag_score=[]

def get_topics(components, feature_names, n=7): #n : 추출한 토픽 개수
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
        print()
        for pp in topic.argsort()[:-n -1:-1]:
#             if(terms[pp]!='스럽다'):

            try:
                tag_idx[terms[pp]]+=1
            except:
                tag_idx[terms[pp]]=1
get_topics(svd_model.components_,terms)

Topic 1: [('후지산', 0.38409), ('있다', 0.37185), ('아름답다', 0.29477), ('좋다', 0.29077), ('에서', 0.2212), ('매우', 0.19757), ('풍경', 0.17812)]

Topic 2: [('좋다', 0.55724), ('풍경', 0.34815), ('아름답다', 0.26603), ('재미', 0.20411), ('아주', 0.15197), ('정말', 0.15114), ('가성', 0.14562)]

Topic 3: [('아름답다', 0.63132), ('매우', 0.2702), ('정말', 0.18842), ('너무', 0.17494), ('풍경', 0.10045), ('특히', 0.07082), ('멀리', 0.05449)]

Topic 4: [('가치', 0.47178), ('있다', 0.46593), ('방문', 0.25679), ('풍경', 0.16861), ('갈다', 0.15862), ('다시', 0.10962), ('멋지다', 0.08478)]

Topic 5: [('풍경', 0.40934), ('재미', 0.20947), ('상징', 0.2093), ('세계', 0.19327), ('비용', 0.16526), ('가장', 0.16205), ('효율', 0.15426)]

Topic 6: [('풍경', 0.42528), ('재미', 0.31685), ('가성', 0.21161), ('비용', 0.16679), ('후지산', 0.15821), ('효율', 0.15255), ('없다', 0.1384)]

Topic 7: [('매우', 0.60274), ('않다', 0.14923), ('명소', 0.1473), ('편안하다', 0.14045), ('재미', 0.1263), ('이며', 0.12146), ('상징', 0.11576)]

Topic 8: [('아주', 0.36911), ('에는', 0.26436), ('있다', 0.2607), ('기회', 0.14806), ('당신', 0

In [61]:
sorted(tag_idx.items(),key=operator.itemgetter(1),reverse=True)[:20]

[('흥미롭다', 21),
 ('스럽다', 20),
 ('이해', 18),
 ('깔끔하다', 18),
 ('지난', 18),
 ('갈다', 16),
 ('어느', 16),
 ('가보다', 16),
 ('기회', 15),
 ('가지', 15),
 ('불행하다', 15),
 ('권장', 15),
 ('상쾌하다', 15),
 ('동경', 15),
 ('특히', 14),
 ('포인트', 14),
 ('하얗다', 14),
 ('가족', 14),
 ('주차', 14),
 ('편안하다', 13)]